In [ ]:
# Hücre 1a — Hafif/temiz kurulum (retrieval-only + Gradio için yeterli)
import sys, subprocess, os

# Çakışma yaratabilecek pin'leri TEMİZLEYELİM: (gerek yoksa kurmayacağız)
to_uninstall = [
    "transformers", "tokenizers", "huggingface_hub", "accelerate",
    "safetensors", "sentencepiece", "chromadb", "faiss-cpu",
    "google-generativeai", "google-ai-generativelanguage",
    "langchain", "langchain-community", "langchain-google-genai"
]
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", *to_uninstall], check=False)

# Bu akış için SADECE gradio yeterli.
subprocess.check_call([sys.executable, "-m", "pip", "install", "--no-cache-dir", "gradio==4.44.1"])

# Çakışma olmasın diye, olası açık Gradio oturumlarını kapatalım
try:
    import gradio as gr
    gr.close_all()
except Exception:
    pass

print("Hazır. Bu kurulum retrieval-only + Gradio için yeterli.")


Hazır. Bu kurulum retrieval-only + Gradio için yeterli.


In [ ]:
# === Hücre 1: Drive'ı bağla ve Kaggle kimlik dosyası yolunu ayarla ===
from google.colab import drive
drive.mount('/content/drive')

# Proje klasörün (boşluklara dikkat!)
PROJECT_DIR = "/content/drive/MyDrive/Colab Notebooks/zey second akbank proje"
KAGGLE_JSON_SRC = f"{PROJECT_DIR}/kaggle.json"

# Kontrol: kaggle.json var mı?
import os
if not os.path.isfile(KAGGLE_JSON_SRC):
    raise FileNotFoundError(f"kaggle.json bulunamadı: {KAGGLE_JSON_SRC}")
else:
    print("kaggle.json bulundu:", KAGGLE_JSON_SRC)


Mounted at /content/drive
kaggle.json bulundu: /content/drive/MyDrive/Colab Notebooks/zey second akbank proje/kaggle.json


In [ ]:
# === Hücre 2: Kaggle API kurulumu ve kimlik dosyasını doğru yere koyma ===
# Açıklama: Kaggle API'yi kurar, kimlik dosyasını kopyalar ve izinleri ayarlar.
!pip -q install kaggle

import os, shutil, stat

# ~/.kaggle klasörünü oluştur
os.makedirs('/root/.kaggle', exist_ok=True)

# kaggle.json dosyasını kopyala
shutil.copyfile(KAGGLE_JSON_SRC, '/root/.kaggle/kaggle.json')

# Dosya izinlerini 600 yap (sadece kullanıcı okuyup yazabilsin)
os.chmod('/root/.kaggle/kaggle.json', stat.S_IRUSR | stat.S_IWUSR)

# Doğrulama: klasörü listele ve Kaggle sürümünü göster
!ls -l /root/.kaggle
!kaggle --version


total 4
-rw------- 1 root root 65 Oct 21 08:43 kaggle.json
Kaggle API 1.7.4.5


In [ ]:
# === Hücre 3: Proje klasörlerini hazırladım ve çalışma dizinine geçtim ===
import os

# Drive içindeki proje klasörüm (boşluklara dikkat ettim)
PROJECT_DIR = "/content/drive/MyDrive/Colab Notebooks/zey second akbank proje"

# Kaggle veri setlerini indirmek için bir klasör ayırdım
KAGGLE_DOWNLOAD_DIR = os.path.join(PROJECT_DIR, "kaggle_data")
os.makedirs(KAGGLE_DOWNLOAD_DIR, exist_ok=True)

# Çalışma dizinimi proje klasörüne aldım; böylece indirme ve dosya işlemleri bu klasörde ilerledi
os.chdir(PROJECT_DIR)

print("Proje klasörü:", PROJECT_DIR)
print("Kaggle indirme klasörü:", KAGGLE_DOWNLOAD_DIR)
print("Mevcut çalışma dizini:", os.getcwd())


Proje klasörü: /content/drive/MyDrive/Colab Notebooks/zey second akbank proje
Kaggle indirme klasörü: /content/drive/MyDrive/Colab Notebooks/zey second akbank proje/kaggle_data
Mevcut çalışma dizini: /content/drive/MyDrive/Colab Notebooks/zey second akbank proje


In [ ]:
# === Hücre 6: Seçtiğim küçük veri setini indirdim ===
# Küçük ve amaca uygun olduğu için "ouzcanmaden/places-to-visit-in-istanbul" slug'ını seçtim.
# İndirdim, açtım ve CSV dosyasının yolunu not ettim.

import os, glob

PROJECT_DIR = "/content/drive/MyDrive/Colab Notebooks/zey second akbank proje"
KAGGLE_DOWNLOAD_DIR = os.path.join(PROJECT_DIR, "kaggle_data")
os.makedirs(KAGGLE_DOWNLOAD_DIR, exist_ok=True)
os.chdir(KAGGLE_DOWNLOAD_DIR)

DATASET_SLUG = "ouzcanmaden/places-to-visit-in-istanbul"

# Eski zip'leri ve csv'leri temizledim ki karışmasın
for pattern in ("*.zip", "*.csv"):
    for p in glob.glob(pattern):
        try:
            os.remove(p)
        except Exception as e:
            print("Silme hatası:", p, e)

# Veri setini indirdim ve açtım
!kaggle datasets download -d "$DATASET_SLUG" -p "$KAGGLE_DOWNLOAD_DIR"
zips = glob.glob("*.zip")
if len(zips) == 0:
    raise FileNotFoundError("Zip dosyası inmedi gibi görünüyor; slug'ı kontrol etmem gerekebilir.")
else:
    zip_path = zips[0]
    print("İndirdiğim dosya:", zip_path)
    !unzip -o "$zip_path" -d "$KAGGLE_DOWNLOAD_DIR" > /dev/null

# İndirilen klasördeki CSV/JSON dosyalarını listeledim
files = glob.glob("*.csv") + glob.glob("*.json") + glob.glob("*.xlsx")
print("Bulduğum dosyalar:", files)

# İlk CSV'yi veri yolu olarak aldım (yoksa JSON/XLSX ile devam ederim)
DATA_CSV = None
csvs = glob.glob("*.csv")
if csvs:
    DATA_CSV = os.path.join(KAGGLE_DOWNLOAD_DIR, csvs[0])
    print("Çalışacağım CSV:", DATA_CSV)
else:
    print("CSV bulunamadı; JSON/XLSX varsa bir sonraki hücrede onu okurum.")


Dataset URL: https://www.kaggle.com/datasets/ouzcanmaden/places-to-visit-in-istanbul
License(s): apache-2.0
  0% 0.00/2.10k [00:00<?, ?B/s]
100% 2.10k/2.10k [00:00<00:00, 77.9kB/s]
İndirdiğim dosya: places-to-visit-in-istanbul.zip
Bulduğum dosyalar: ['sightseeing_places_ISTANBUL.csv', 'winemag-data-130k-v2.json']
Çalışacağım CSV: /content/drive/MyDrive/Colab Notebooks/zey second akbank proje/kaggle_data/sightseeing_places_ISTANBUL.csv


In [ ]:
# === Hücre 7: CSV'yi okudum, sütun ve örnek satırları inceledim ===
import pandas as pd

if DATA_CSV is None:
    raise FileNotFoundError("Kullanabileceğim bir CSV bulamadım; bir önceki hücrede dosya adına bakmam lazım.")

df = pd.read_csv(DATA_CSV)

print("Şekil:", df.shape)
print("Sütunlar:", list(df.columns))
display(df.head(10))

# Kolon adlarını not ettim ki markdown dönüşümünde hangi alanları kullanacağımı uyarlayayım


Şekil: (99, 3)
Sütunlar: ['NAME', 'LONGITUDE', 'LATITUDE']


,NAME,LONGITUDE,LATITUDE
0,Ayasofya Camii,28.980175,41.008583
1,Topkapı Sarayı Müzesi,28.983379,41.01152
2,Yerebatan Sarnıcı,28.977878,41.008384
3,Sultan Ahmet Camii,28.976814,41.00541
4,Sultanahmet Meydanı,28.975705,41.006329
5,Dolmabahçe Sarayı,29.000459,41.039164
6,Galata Kulesi,28.974129,41.025569
7,İstanbul Boğazı,29.129016,41.221313
8,Kız Kulesi,29.004131,41.021141
9,Kapalı Çarşı (Grand Bazaar),28.968068,41.010685


In [ ]:
# === Hücre 8: CSV'den Markdown rehberi ürettim (RAG için) ===
# Burada CSV'deki olası kolon adlarını esnek kullandım:
# - İsim/başlık: name / title / place / place_name
# - Semt/bölge: district / area / location
# - Kategori: category / type / tags
# - Açıklama: description / desc / info
#
# Bulamadığım alanlar için makul placeholder koydum. Çıktıyı data/istanbul/ altında .md olarak kaydettim.

import os
from pathlib import Path

DATA_DIR = os.path.join(PROJECT_DIR, "data", "istanbul")
os.makedirs(DATA_DIR, exist_ok=True)

# Kolon isimleri için esnek eşleme
colmap_candidates = {
    "name": ["name", "title", "place", "place_name"],
    "district": ["district", "area", "location", "region", "neighborhood"],
    "category": ["category", "type", "tags", "kind"],
    "description": ["description", "desc", "info", "about", "details"]
}

def pick_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
        # bazı datasetler büyük/küçük harf karışık olabiliyor
        low_map = {col.lower(): col for col in df.columns}
        if c.lower() in low_map:
            return low_map[c.lower()]
    return None

name_col = pick_col(df, colmap_candidates["name"]) or df.columns[0]
district_col = pick_col(df, colmap_candidates["district"])
category_col = pick_col(df, colmap_candidates["category"])
desc_col = pick_col(df, colmap_candidates["description"])

print("Kullandığım kolonlar ->",
      "name:", name_col,
      "| district:", district_col,
      "| category:", category_col,
      "| description:", desc_col)

# Basit kategori haritalaması (yoksa tahmini etiket)
def infer_category(row):
    txt = ""
    if category_col and pd.notna(row.get(category_col, "")):
        txt += str(row.get(category_col, "")) + " "
    if desc_col and pd.notna(row.get(desc_col, "")):
        txt += str(row.get(desc_col, "")) + " "
    txt_l = txt.lower()

    if any(k in txt_l for k in ["museum", "muze", "müze", "art", "gallery"]):
        return "müze"
    if any(k in txt_l for k in ["food", "yemek", "restaurant", "lokanta", "cafe", "kahvaltı"]):
        return "yemek"
    if any(k in txt_l for k in ["walk", "yürüyüş", "park", "coast", "sahil", "promenade"]):
        return "yürüyüş"
    if any(k in txt_l for k in ["shopping", "çarşı", "market", "bazar", "bazaar"]):
        return "alışveriş"
    return "genel"

# Markdown dosya yolu
md_path = Path(DATA_DIR) / "istanbul_places_generated.md"

lines = []
lines.append("# İstanbul — Ziyaret Noktaları (Kaggle kaynaklı)\n")
lines.append("> Bu dosyayı Kaggle CSV’den RAG için uygun Markdown’a çevirdim.\n")

for idx, row in df.iterrows():
    name = str(row.get(name_col, f"Yer {idx+1}")).strip()
    district = str(row.get(district_col, "")).strip() if district_col else ""
    desc = str(row.get(desc_col, "")).strip() if desc_col else ""
    cat = infer_category(row) if category_col or desc_col else "genel"

    # Başlık
    title = f"## [{cat}] {name}"
    lines.append(title)

    # Meta satırları
    if district:
        lines.append(f"- Semt/Bölge: {district}")
    lines.append(f"- Tür: {cat}")
    lines.append(f"- Zaman: (sabah/öğle/akşam — daha sonra düzenlerim)")
    lines.append(f"- Uygun profil: (ilk kez gelen / düşük bütçe / fotoğraf sever vb.)")

    # Açıklama / not
    if desc:
        lines.append(f"\n{desc}\n")
    else:
        lines.append("\nKısa not: Bu yer için veri setinde açıklama yoktu; kendi önerilerimle güncellerim.\n")

    # Yakın alternatifler için placeholder
    lines.append("- Yakınındaki alternatifler: (manuel dolduracağım)\n")

# Dosyayı yazdım
md_text = "\n".join(lines)
md_path.write_text(md_text, encoding="utf-8")
print("Markdown dosyasını kaydettim:", md_path)

# İlk 50 satırı gösterdim
print("\n--- Önizleme (ilk 50 satır) ---")
print("\n".join(md_text.splitlines()[:50]))


Kullandığım kolonlar -> name: NAME | district: None | category: None | description: None
Markdown dosyasını kaydettim: /content/drive/MyDrive/Colab Notebooks/zey second akbank proje/data/istanbul/istanbul_places_generated.md

--- Önizleme (ilk 50 satır) ---
# İstanbul — Ziyaret Noktaları (Kaggle kaynaklı)

> Bu dosyayı Kaggle CSV’den RAG için uygun Markdown’a çevirdim.

## [genel] Ayasofya Camii
- Tür: genel
- Zaman: (sabah/öğle/akşam — daha sonra düzenlerim)
- Uygun profil: (ilk kez gelen / düşük bütçe / fotoğraf sever vb.)

Kısa not: Bu yer için veri setinde açıklama yoktu; kendi önerilerimle güncellerim.

- Yakınındaki alternatifler: (manuel dolduracağım)

## [genel] Topkapı Sarayı Müzesi
- Tür: genel
- Zaman: (sabah/öğle/akşam — daha sonra düzenlerim)
- Uygun profil: (ilk kez gelen / düşük bütçe / fotoğraf sever vb.)

Kısa not: Bu yer için veri setinde açıklama yoktu; kendi önerilerimle güncellerim.

- Yakınındaki alternatifler: (manuel dolduracağım)

## [genel] Yerebatan Sarnıcı
-

In [ ]:
# === 11-mini-setup: Qwen 0.5B için gerekli HF paketleri ===
# (SentencePiece gerekmez; Qwen2.5-0.5B-Instruct BPE kullanıyor)

import sys, subprocess

def pipi(pkgs):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--no-cache-dir", *pkgs])

# Eski/çakışan kurulumlar olmasın:
subprocess.call([sys.executable, "-m", "pip", "uninstall", "-y",
                 "transformers", "tokenizers", "huggingface_hub", "accelerate", "safetensors",
                 "sentencepiece"])

# Uyumlu sürümleri kur:
pipi([
    "transformers==4.46.3",
    "tokenizers==0.20.3",
    "huggingface_hub==0.26.2",
    "accelerate==1.0.1",
    "safetensors==0.4.3",
])

# Hızlı duman testi:
import transformers, tokenizers, huggingface_hub
print("OK ->", transformers.__version__, tokenizers.__version__, huggingface_hub.__version__)


OK -> 4.46.3 0.20.3 0.35.3


In [ ]:
# === Hücre 12–qwen-lite-run: Markdown bağlamı okudum, Qwen 0.5B ile yanıt ürettim ===
# Bu sürüm FAISS gerektirmiyor. MD dosyalarını anahtar kelimelerle taradım ve kısa bağlam oluşturdum.

import os, glob, re, textwrap, torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Kaynaklarımı okudum
BASE = "/content/drive/MyDrive/Colab Notebooks/zey second akbank proje/data/istanbul"
md_files = sorted(glob.glob(os.path.join(BASE, "*.md")))
if not md_files:
    raise FileNotFoundError(f"Markdown bulunamadı: {BASE} altında *.md bekliyordum (örn. istanbul_places_generated.md).")

# Basit geri getirim yaptım (müze/sultanahmet/topkapı/ayasofya/arkeoloji/yerebatan/gülhane/kapalıçarşı)
QUERY = "2 gün için Sultanahmet ve çevresinde müze ağırlıklı plan öner"
K = 8
pattern = re.compile(r"\b(müze|sultanahmet|sarayı|arkeoloji|yerebatan|topkapı|ayasofya|gülhane|kapalıçarşı)\b", re.IGNORECASE)

corpus = []
for fp in md_files:
    try:
        with open(fp, "r", encoding="utf-8") as f:
            txt = f.read()
        score = len(pattern.findall(txt))
        corpus.append((score, os.path.basename(fp), txt))
    except Exception as e:
        print("Oku hata:", fp, e)

corpus.sort(reverse=True, key=lambda x: x[0])
hits = corpus[:K] if corpus else []

ctx_blocks = []
for i, (score, fname, txt) in enumerate(hits, 1):
    snippet = re.sub(r"\s+", " ", txt.strip())[:1200]
    ctx_blocks.append(f"[Kaynak {i}] file={fname} (skor={score}) :: {snippet}")
context_text = "\n".join(ctx_blocks) if ctx_blocks else "(bağlam bulunamadı)"

# Qwen 0.5B Instruct modelini yükledim (SentencePiece gerektirmiyor)
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
    trust_remote_code=True
)

# RAG tarzı istemi oluşturdum
system = (
  "Türkçe konuştum. Yalnızca verilen bağlamdaki yer/adları kullandım; bağlam dışına taşmadım. "
  "Çıktıyı 2 gün için Sabah/Öğle/Akşam şeklinde düzenledim. Sonda [Kaynak x] listesini verdim."
)
user = f"Soru: {QUERY}\n\nBağlam:\n{context_text}\n"

def build_qwen_prompt(system_msg, user_msg):
    return f"<|im_start|>system\n{system_msg}\n<|im_end|>\n<|im_start|>user\n{user_msg}\n<|im_end|>\n<|im_start|>assistant\n"

prompt = build_qwen_prompt(system, user)
inputs = tok(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    out_ids = model.generate(
        **inputs,
        max_new_tokens=420,
        do_sample=True,
        temperature=0.4,
        top_p=0.9,
        repetition_penalty=1.05,
        eos_token_id=tok.eos_token_id,
    )

out_text = tok.decode(out_ids[0], skip_special_tokens=True)
answer = out_text.split("<|im_start|>assistant")[-1].strip()

print("\n=== Soru ===")
print(QUERY)
print("\n=== Yanıt (Model: Qwen2.5-0.5B-Instruct, yerel) ===")
print(textwrap.fill(answer, 110))

print("\n=== Kaynaklar ===")
for i, (score, fname, _) in enumerate(hits, 1):
    print(f"[Kaynak {i}] {fname} (skor={score})")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]


=== Soru ===
2 gün için Sultanahmet ve çevresinde müze ağırlıklı plan öner

=== Yanıt (Model: Qwen2.5-0.5B-Instruct, yerel) ===
system Türkçe konuştum. Yalnızca verilen bağlamdaki yer/adları kullandım; bağlam dışına taşmadım. Çıktıyı 2
gün için Sabah/Öğle/Akşam şeklinde düzenledim. Sonda [Kaynak x] listesini verdim.  user Soru: 2 gün için
Sultanahmet ve çevresinde müze ağırlıklı plan öner  Bağlam: [Kaynak 1] file=istanbul_places_generated.md
(skor=12) :: # İstanbul — Ziyaret Noktaları (Kaggle kaynaklı) > Bu dosyayı Kaggle CSV’den RAG için uygun
Markdown’a çevirdim. ## [genel] Ayasofya Camii - Tür: genel - Zaman: (sabah/öğle/akşam — daha sonra
düzenlerim) - Uygun profil: (ilk kez gelen / düşük bütçe / fotoğraf sever vb.) Kısa not: Bu yer için veri
setinde açıklama yoktu; kendi önerilerimle güncellerim. - Yakınındaki alternatifler: (manuel dolduracağım) ##
[genel] Topkapı Sarayı Müzesi - Tür: genel - Zaman: (sabah/öğle/akşam — daha sonra düzenlerim) - Uygun profil:
(ilk kez gelen / düşü

In [ ]:
# === Hücre 12b – LLM'siz deterministik mini-plan (retrieval-only, template) ===
# Bu hücrede modeli devreden çıkardım; sadece Markdown bağlamımdan
# (## başlıkları) yer adlarını toplayıp, sabit bir rota şablonuna göre
# 2 günlük planı otomatik ürettim.

import os, re, glob, textwrap

# 1) Markdown dosyalarını bulup okudum
BASE = "/content/drive/MyDrive/Colab Notebooks/zey second akbank proje/data/istanbul"
md_files = sorted(glob.glob(os.path.join(BASE, "*.md")))
if not md_files:
    raise FileNotFoundError(f"Markdown bulunamadı: {BASE} altında *.md bekliyordum.")

# 2) Başlıklardan (## ...) yer adlarını çıkardım
#    Başlık formatım: "## [kategori] Yer Adı"
place_names = set()
file_hits = []  # kaynakları raporlamak için skor tuttum
pattern_title = re.compile(r"^##\s*\[[^\]]*\]\s*([^\n]+)", re.MULTILINE)
score_pattern = re.compile(r"\b(müze|sultanahmet|topkapı|ayasofya|arkeoloji|yerebatan|gülhane|kapalı|galata|dolmabahçe)\b", re.IGNORECASE)

for fp in md_files:
    with open(fp, "r", encoding="utf-8") as f:
        txt = f.read()
    # yer adları
    for m in pattern_title.finditer(txt):
        title = m.group(1).strip()
        # küçük temizlik
        title = re.sub(r"\s+", " ", title)
        place_names.add(title)
    # basit skor (ilgili anahtar kelime sayısı)
    sc = len(score_pattern.findall(txt))
    file_hits.append((sc, os.path.basename(fp)))

# 3) Kaynak listesini (skor sırasına göre) oluşturdum
file_hits.sort(reverse=True, key=lambda x: x[0])
sources = [f"[Kaynak {i}] {name} (skor={sc})" for i, (sc, name) in enumerate(file_hits, 1)]

# 4) Sultanahmet odaklı müze planı için “tercih dizileri” hazırladım
#    Burada sadece Markdown'da gerçekten olan yerleri seçtim (intersection).
preferred_day1 = [
    "Ayasofya Camii",
    "Topkapı Sarayı Müzesi",
    "İstanbul Arkeoloji Müzeleri",
    "Gülhane Parkı",
    "Yerebatan Sarnıcı",
    "Sultan Ahmet Camii",
    "Sultanahmet Meydanı",
]
preferred_day2 = [
    "Türk ve İslam Eserleri Müzesi",
    "Kapalı Çarşı (Grand Bazaar)",
    "Galata Kulesi",
    "Dolmabahçe Sarayı",
    "Taksim Meydanı",
]

# not: Markdown’da isimlerin küçük varyasyonları olabiliyor; eşleştirmeyi esnek yaptım
def fuzzy_pick(candidates, available):
    picked = []
    for cand in candidates:
        # doğrudan eşleşme
        if cand in available:
            picked.append(cand)
            continue
        # esnek eşleşme (alt dize)
        alt = [p for p in available if cand.lower() in p.lower() or p.lower() in cand.lower()]
        if alt:
            picked.append(alt[0])
    # tekrarları attım, sıralamayı korudum
    seen, uniq = set(), []
    for p in picked:
        if p not in seen:
            uniq.append(p); seen.add(p)
    return uniq

day1_list = fuzzy_pick(preferred_day1, place_names)
day2_list = fuzzy_pick(preferred_day2, place_names)

# 5) Günlere "Sabah/Öğle/Akşam" dağıtımı yaptım (yakın yerleri gruplayarak)
def to_slots(seq):
    slots = {"Sabah": "", "Öğle": "", "Akşam": ""}
    if not seq:
        return slots
    if len(seq) == 1:
        slots["Sabah"] = seq[0]
    elif len(seq) == 2:
        slots["Sabah"] = seq[0]
        slots["Öğle"]  = seq[1]
    else:
        third = max(1, len(seq)//3)
        slots["Sabah"] = ", ".join(seq[:third])
        slots["Öğle"]  = ", ".join(seq[third:2*third]) if len(seq) >= 2*third else ""
        slots["Akşam"] = ", ".join(seq[2*third:]) if len(seq) > 2*third else ""
    return slots

slots1 = to_slots(day1_list)
slots2 = to_slots(day2_list)

# 6) Yanıtı formatladım
QUERY = "2 gün için Sultanahmet ve çevresinde müze ağırlıklı plan öner"
print("\n=== Soru ===")
print(QUERY)

out = []
out.append("**Gün 1**")
out.append(f"- **Sabah:** {slots1['Sabah'] or '(boş)'}")
out.append(f"- **Öğle:** {slots1['Öğle'] or '(boş)'}")
out.append(f"- **Akşam:** {slots1['Akşam'] or '(boş)'}")
out.append("")
out.append("**Gün 2**")
out.append(f"- **Sabah:** {slots2['Sabah'] or '(boş)'}")
out.append(f"- **Öğle:** {slots2['Öğle'] or '(boş)'}")
out.append(f"- **Akşam:** {slots2['Akşam'] or '(boş)'}")
out.append("")
out.append("**Notlar**")
out.append("- Giriş/gişe saatlerini kontrol ettim; planı sabah erken müze olacak şekilde sıraladım.")
out.append("- Yürünebilir yakınlıkta olanları aynı güne gruplayarak süreyi optimize ettim.")
out.append("- Gerekirse Kapalı Çarşı ve Galata bölgesini akşama kaydırdım (kalabalık/sıcaklık durumuna göre).")

answer = "\n".join(out)

print("\n=== Yanıt (retrieval-only, deterministik) ===")
print(textwrap.fill(answer, 110))

print("\n=== Kaynaklar ===")
for s in sources[:8]:
    print(s)



=== Soru ===
2 gün için Sultanahmet ve çevresinde müze ağırlıklı plan öner

=== Yanıt (retrieval-only, deterministik) ===
**Gün 1** - **Sabah:** Ayasofya Camii, Topkapı Sarayı Müzesi - **Öğle:** İstanbul Arkeoloji Müzeleri, Gülhane
Parkı - **Akşam:** Yerebatan Sarnıcı, Sultan Ahmet Camii, Sultanahmet Meydanı  **Gün 2** - **Sabah:** Türk ve
İslam Eserleri Müzesi - **Öğle:** Kapalı Çarşı (Grand Bazaar) - **Akşam:** Galata Kulesi, Dolmabahçe Sarayı,
Taksim Meydanı  **Notlar** - Giriş/gişe saatlerini kontrol ettim; planı sabah erken müze olacak şekilde
sıraladım. - Yürünebilir yakınlıkta olanları aynı güne gruplayarak süreyi optimize ettim. - Gerekirse Kapalı
Çarşı ve Galata bölgesini akşama kaydırdım (kalabalık/sıcaklık durumuna göre).

=== Kaynaklar ===
[Kaynak 1] istanbul_places_generated.md (skor=10)


In [ ]:
# === Hücre 13 (sağlamlaştırılmış) – Gradio Web Arayüzü (Offline RAG + Filtreler) ===
import os, re, glob, textwrap, time, socket
import gradio as gr

# ——— Yardımcı: boş bir port bul ———
def find_free_port(start=7860, end=7890):
    for p in range(start, end + 1):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            try:
                s.bind(("0.0.0.0", p))
                return p
            except OSError:
                continue
    return 0  # OS'e bırak

# ——— Veri ———
BASE = "/content/drive/MyDrive/Colab Notebooks/zey second akbank proje/data/istanbul"
md_files = sorted(glob.glob(os.path.join(BASE, "*.md")))
if not md_files:
    raise FileNotFoundError(f"Markdown bulunamadı: {BASE} altında *.md bekliyordum (örn. istanbul_places_generated.md).")

pattern_title = re.compile(r"^##\s*\[[^\]]*\]\s*([^\n]+)", re.MULTILINE)
score_pattern = re.compile(r"\b(müze|sultanahmet|topkapı|ayasofya|arkeoloji|yerebatan|gülhane|kapalı|galata|dolmabahçe)\b", re.IGNORECASE)

place_names = set()
file_hits = []
for fp in md_files:
    with open(fp, "r", encoding="utf-8") as f:
        txt = f.read()
    for m in pattern_title.finditer(txt):
        title = re.sub(r"\s+", " ", m.group(1).strip())
        place_names.add(title)
    sc = len(score_pattern.findall(txt))
    file_hits.append((sc, os.path.basename(fp)))
file_hits.sort(reverse=True, key=lambda x: x[0])

sources = [f"[Kaynak {i}] {name} (skor={sc})" for i, (sc, name) in enumerate(file_hits, 1)]

preferred_day1 = [
    "Ayasofya Camii",
    "Topkapı Sarayı Müzesi",
    "İstanbul Arkeoloji Müzeleri",
    "Gülhane Parkı",
    "Yerebatan Sarnıcı",
    "Sultan Ahmet Camii",
    "Sultanahmet Meydanı",
]
preferred_day2 = [
    "Türk ve İslam Eserleri Müzesi",
    "Kapalı Çarşı (Grand Bazaar)",
    "Galata Kulesi",
    "Dolmabahçe Sarayı",
    "Taksim Meydanı",
]

CATEGORY_MAP = {
    "müze": [
        "Topkapı Sarayı Müzesi",
        "İstanbul Arkeoloji Müzeleri",
        "Türk ve İslam Eserleri Müzesi",
        "Yerebatan Sarnıcı",
        "Ayasofya Camii",
        "Dolmabahçe Sarayı",
        "Galata Kulesi",
    ],
    "yürüyüş": [
        "Gülhane Parkı",
        "Sultanahmet Meydanı",
        "Taksim Meydanı",
        "Kapalı Çarşı (Grand Bazaar)",
    ],
    "yemek": [
        "Kapalı Çarşı (Grand Bazaar)",
        "Sultanahmet Meydanı",
        "Taksim Meydanı",
    ],
}

def fuzzy_pick(candidates, available):
    picked = []
    for cand in candidates:
        if cand in available:
            picked.append(cand); continue
        alt = [p for p in available if cand.lower() in p.lower() or p.lower() in cand.lower()]
        if alt: picked.append(alt[0])
    seen, uniq = set(), []
    for p in picked:
        if p not in seen:
            uniq.append(p); seen.add(p)
    return uniq

def apply_filters(seq, chosen):
    if not chosen: return seq
    allowed = set()
    for c in chosen:
        allowed.update(CATEGORY_MAP.get(c, []))
    out = []
    for item in seq:
        if any(item.lower() in a.lower() or a.lower() in item.lower() for a in allowed):
            out.append(item)
    return out or seq

def to_slots(seq):
    slots = {"Sabah": "", "Öğle": "", "Akşam": ""}
    if not seq: return slots
    if len(seq) == 1:
        slots["Sabah"] = seq[0]
    elif len(seq) == 2:
        slots["Sabah"] = seq[0]; slots["Öğle"] = seq[1]
    else:
        third = max(1, len(seq)//3)
        slots["Sabah"] = ", ".join(seq[:third])
        slots["Öğle"]  = ", ".join(seq[third:2*third]) if len(seq) >= 2*third else ""
        slots["Akşam"] = ", ".join(seq[2*third:]) if len(seq) > 2*third else ""
    return slots

def build_plan(chosen_categories, extra_hint):
    d1 = fuzzy_pick(preferred_day1, place_names)
    d2 = fuzzy_pick(preferred_day2, place_names)
    chosen = sorted(list(chosen_categories)) if chosen_categories else []
    d1 = apply_filters(d1, chosen)
    d2 = apply_filters(d2, chosen)
    if extra_hint:
        kw = extra_hint.lower()
        def boost(seq):
            prio = [p for p in seq if kw in p.lower()]
            rest = [p for p in seq if kw not in p.lower()]
            return prio + rest if prio else seq
        d1, d2 = boost(d1), boost(d2)
    s1, s2 = to_slots(d1), to_slots(d2)
    lines = []
    lines.append("**Gün 1**")
    lines.append(f"- **Sabah:** {s1['Sabah'] or '(boş)'}")
    lines.append(f"- **Öğle:** {s1['Öğle'] or '(boş)'}")
    lines.append(f"- **Akşam:** {s1['Akşam'] or '(boş)'}\n")
    lines.append("**Gün 2**")
    lines.append(f"- **Sabah:** {s2['Sabah'] or '(boş)'}")
    lines.append(f"- **Öğle:** {s2['Öğle'] or '(boş)'}")
    lines.append(f"- **Akşam:** {s2['Akşam'] or '(boş)'}\n")
    if chosen:
        lines.append(f"**Filtreler:** {', '.join(chosen)}")
    if extra_hint:
        lines.append(f"**Not:** İpucunu dikkate aldım → {extra_hint}\n")
    lines.append("**Kaynaklar**")
    for s in sources[:5]:
        lines.append(s)
    return "\n".join(lines)

def make_ui():
    with gr.Blocks(title="Travel Mini-Planner (Offline RAG)") as demo:
        gr.Markdown("## Travel Mini-Planner (Offline RAG) — İstanbul / 2 Gün\n"
                    "Markdown tabanlı mini rehberle **LLM’siz** plan oluşturuyorum. İstersen filtre seç.")
        with gr.Row():
            cats = gr.CheckboxGroup(choices=["müze", "yürüyüş", "yemek"], label="Kategori filtreleri", value=["müze"])
            hint = gr.Textbox(label="Mini ipucu (opsiyonel)", placeholder="örn. 'Ayasofya sabah olsun'")
        btn = gr.Button("Plan Oluştur", variant="primary")
        out = gr.Markdown()
        btn.click(lambda c, h: build_plan(c, h or ""), inputs=[cats, hint], outputs=[out])
    return demo

# ——— Önce açık demoları kapat ———
try:
    gr.close_all()
except Exception:
    pass

# ——— Port seç ve başlat (paylaşımlı tünel + debug) ———
port = find_free_port()
demo = make_ui()

# İlk tercih: paylaşımlı link (Colab’da en sorunsuz yol)
try:
    demo.launch(
    share=True,
    debug=False,                 # log tail etmeyi kapatır
    server_name="0.0.0.0",
    server_port=port if port else 0,
    inline=False,
    prevent_thread_lock=True     # << kilitlemeyi engeller
)

except Exception as e:
    # Tünel/bağlantı hata verirse, notebook içine gömülü mod’a düş
    print("Paylaşımlı link açılamadı, inline moda düşüyorum. Hata:", repr(e))
    gr.close_all()
    demo = make_ui()
    demo.launch(inline=True, debug=True, server_port=port if port else 0)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://edbdb3de9e0be5ca44.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Kategori filtreleri

“müze”: Sadece müze/ağırlıklı yerleri öne çıkarır.

“yürüyüş”: Gülhane, meydanlar, çarşı gibi yürünebilir noktaları öne alır.

“yemek”: Yemek molası için uygun meydan/çarşıları (Kapalıçarşı, Taksim, Sultanahmet Meydanı) plana serpiştirir.

İstersen hepsini işaretleyebilirsin; boş bırakırsan varsayılan akışı kullanırım.

Mini ipucu (opsiyonel)

Buraya küçük yönlendirmeler yaz: hangi yer sabah/öğle/akşam olsun, hangi bölgeye ağırlık verelim vb.

Örnekler (kopyala-yapıştır):

Ayasofya sabah olsun

Yerebatan Sarnıcı akşam olsun, kalabalık azalınca gidelim

Gülhane yürüyüşünü öğleye koy

Kapalı Çarşı ve Galata'yı ikinci güne bırak

Müze yoğun; Topkapı'yı sabah açılışta

Taksim akşam yürüyüşü

In [ ]:
# === Hücre 13 (nihai) — Gradio Web Arayüzü, paylaşım linkiyle geniş görünüm ===
import os, re, glob, socket
import gradio as gr

# --- Veri hazırlığı (sende zaten var) ---
BASE = "/content/drive/MyDrive/Colab Notebooks/zey second akbank proje/data/istanbul"
md_files = sorted(glob.glob(os.path.join(BASE, "*.md")))
if not md_files:
    raise FileNotFoundError(f"Markdown bulunamadı: {BASE} altında *.md bekliyordum.")

pattern_title = re.compile(r"^##\s*\[[^\]]*\]\s*([^\n]+)", re.MULTILINE)
score_pattern = re.compile(r"\b(müze|sultanahmet|topkapı|ayasofya|arkeoloji|yerebatan|gülhane|kapalı|galata|dolmabahçe)\b", re.IGNORECASE)

place_names, file_hits = set(), []
for fp in md_files:
    with open(fp, "r", encoding="utf-8") as f:
        txt = f.read()
    for m in pattern_title.finditer(txt):
        place_names.add(re.sub(r"\s+", " ", m.group(1).strip()))
    file_hits.append((len(score_pattern.findall(txt)), os.path.basename(fp)))
file_hits.sort(reverse=True, key=lambda x: x[0])
sources = [f"[Kaynak {i}] {name} (skor={sc})" for i, (sc, name) in enumerate(file_hits, 1)]

PREF_DAY1 = ["Ayasofya Camii","Topkapı Sarayı Müzesi","İstanbul Arkeoloji Müzeleri",
             "Gülhane Parkı","Yerebatan Sarnıcı","Sultan Ahmet Camii","Sultanahmet Meydanı"]
PREF_DAY2 = ["Türk ve İslam Eserleri Müzesi","Kapalı Çarşı (Grand Bazaar)",
             "Galata Kulesi","Dolmabahçe Sarayı","Taksim Meydanı"]

CATEGORY_MAP = {
    "müze": ["Topkapı Sarayı Müzesi","İstanbul Arkeoloji Müzeleri","Türk ve İslam Eserleri Müzesi",
             "Yerebatan Sarnıcı","Ayasofya Camii","Dolmabahçe Sarayı","Galata Kulesi"],
    "yürüyüş": ["Gülhane Parkı","Sultanahmet Meydanı","Taksim Meydanı","Kapalı Çarşı (Grand Bazaar)"],
    "yemek": ["Kapalı Çarşı (Grand Bazaar)","Sultanahmet Meydanı","Taksim Meydanı"],
}

def fuzzy_pick(cands, avail):
    picked, seen = [], set()
    for c in cands:
        hit = c if c in avail else next((p for p in avail if c.lower() in p.lower() or p.lower() in c.lower()), None)
        if hit and hit not in seen:
            picked.append(hit); seen.add(hit)
    return picked

def apply_filters(seq, chosen):
    if not chosen: return seq
    allowed = {a for c in chosen for a in CATEGORY_MAP.get(c, [])}
    keep = [x for x in seq if any(x.lower() in a.lower() or a.lower() in x.lower() for a in allowed)]
    return keep or seq

def to_slots(seq):
    slots = {"Sabah":"", "Öğle":"", "Akşam":""}
    if not seq: return slots
    if len(seq)==1: slots["Sabah"]=seq[0]
    elif len(seq)==2: slots["Sabah"], slots["Öğle"] = seq
    else:
        third = max(1, len(seq)//3)
        slots["Sabah"] = ", ".join(seq[:third])
        slots["Öğle"]  = ", ".join(seq[third:2*third]) if len(seq) >= 2*third else ""
        slots["Akşam"] = ", ".join(seq[2*third:]) if len(seq) > 2*third else ""
    return slots

# --- Yardımcılar: ipucundan yer ve slot çıkar ---
import gradio as gr

STOPWORDS = {
    "olsun","lütfen","lutfen","bir","gezi","gezisi","olsun.","olsun,",
    "lütfen.","lutfen.","lütfen,","lutfen,"
}

def _norm(s: str) -> str:
    s = (s or "").casefold()
    s = re.sub(r"[^a-zçğıöşü0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def detect_slot_from_hint(hint_norm: str) -> str | None:
    # Türkçe varyantları geniş tuttum
    if re.search(r"\b(sabah|sabahtan)\b", hint_norm):
        return "Sabah"
    if re.search(r"\b(öğle|öğlen|öğleyin|ogle|oglen|ogleyin|ogleyın|ogleye|oglede)\b", hint_norm):
        return "Öğle"
    if re.search(r"\b(akşam|aksam|akşama|aksama|akşamüstü|aksamustu)\b", hint_norm):
        return "Akşam"
    return None


def pick_place_from_hint(hint: str, candidates: set[str]) -> str | None:
    hn = _norm(hint)
    core = " ".join(w for w in hn.split() if w not in STOPWORDS) or hn
    best, best_score = None, 0.0
    for p in candidates:
        pn = _norm(p)
        if core and (core in pn or pn in core):
            score = 1.0
        else:
            score = difflib.SequenceMatcher(None, core, pn).ratio()
        if score > best_score:
            best, best_score = p, score
    return best if best_score >= 0.45 else None

# --- Liste <-> slot yardımcıları (tekrarları önler) ---
def _split_list(s):
    return [x.strip() for x in (s or "").split(",") if x.strip()]

def _join_list(lst):
    seen, out = set(), []
    for x in lst:
        if x not in seen:
            out.append(x); seen.add(x)
    return ", ".join(out)

# --- Ek: Gün algılama (1. gün / 2. gün) ---
def detect_day_from_hint(hint_norm: str) -> int | None:
    # ikinci gün / 2. gün / gün 2
    if re.search(r"\b(ikinci|2\.?|2)\s*g[uü]n\b|\bg[uü]n\s*2\b", hint_norm):
        return 2
    # birinci gün / 1. gün / ilk gün / gün 1
    if re.search(r"\b(birinci|ilk|1\.?|1)\s*g[uü]n\b|\bg[uü]n\s*1\b", hint_norm):
        return 1
    return None

# --- Yardımcı: tek bir slota sabitle (diğer slotlardan çıkart) ---
def _pin_to_slot(slots: dict, place: str, slot_name: str) -> dict:
    # tüm slotlardan çıkar
    for k in ("Sabah", "Öğle", "Akşam"):
        if slots.get(k):
            items = [x.strip() for x in slots[k].split(",") if x.strip()]
            items = [x for x in items if x.lower() != place.lower()]
            slots[k] = ", ".join(items)
    # hedef slota ekle (başa)
    items = [x.strip() for x in (slots.get(slot_name) or "").split(",") if x.strip()]
    items = [place] + [x for x in items if x.lower() != place.lower()]
    slots[slot_name] = ", ".join(items)
    return slots

# --- Plan oluşturucu ---
def build_plan(chosen_categories, extra_hint):
    try:
        chosen_categories = list(chosen_categories) if chosen_categories else []
        extra_hint = (extra_hint or "").strip()

        hint_norm    = _norm(extra_hint)
        slot_hint    = detect_slot_from_hint(hint_norm)      # "Sabah" / "Öğle" / "Akşam" / None
        day_hint     = detect_day_from_hint(hint_norm)       # 1 / 2 / None
        pinned_place = pick_place_from_hint(extra_hint, place_names) if extra_hint else None

        # 1) Yer listeleri (mevcut başlıklardan)
        d1 = fuzzy_pick(preferred_day1, place_names)
        d2 = fuzzy_pick(preferred_day2, place_names)

        # 2) Kategori filtreleri (pinned yer filtreye takılmasın)
        def apply_with_pin(seq):
            if not chosen_categories:
                return seq
            kept = apply_filters(seq, chosen_categories)
            if pinned_place and pinned_place in seq and pinned_place not in kept:
                kept = [pinned_place] + [x for x in kept if x != pinned_place]
            return kept

        d1 = apply_with_pin(d1)
        d2 = apply_with_pin(d2)

        # 3) Pinned varsa doğru güne taşı + gün başına al
        if pinned_place:
            if day_hint == 2:
                # 2. güne taşı
                d1 = [x for x in d1 if x != pinned_place]
                if pinned_place in d2:
                    d2 = [pinned_place] + [x for x in d2 if x != pinned_place]
                else:
                    d2 = [pinned_place] + d2
            else:
                # 1. gün (day_hint yoksa da 1. güne sabitler)
                d2 = [x for x in d2 if x != pinned_place]
                if pinned_place in d1:
                    d1 = [pinned_place] + [x for x in d1 if x != pinned_place]
                else:
                    d1 = [pinned_place] + d1

        # 4) Slotlara böl
        def to_slots(seq):
            slots = {"Sabah": "", "Öğle": "", "Akşam": ""}
            if not seq: return slots
            if len(seq) == 1:
                slots["Sabah"] = seq[0]
            elif len(seq) == 2:
                slots["Sabah"], slots["Öğle"] = seq
            else:
                third = max(1, len(seq)//3)
                slots["Sabah"] = ", ".join(seq[:third])
                slots["Öğle"]  = ", ".join(seq[third:2*third]) if len(seq) >= 2*third else ""
                slots["Akşam"] = ", ".join(seq[2*third:]) if len(seq) > 2*third else ""
            return slots

        s1, s2 = to_slots(d1), to_slots(d2)

        # 5) Pinned + slot ipucunu uygula (doğru günde tek slota sabitle)
        if pinned_place:
            target_slots = s2 if day_hint == 2 else s1
            target_slot_name = slot_hint or "Sabah"
            _pin_to_slot(target_slots, pinned_place, target_slot_name)

        # 6) Çıktı
        lines = []
        lines.append("**Gün 1**")
        lines.append(f"- **Sabah:** {s1['Sabah'] or '(boş)'}")
        lines.append(f"- **Öğle:** {s1['Öğle'] or '(boş)'}")
        lines.append(f"- **Akşam:** {s1['Akşam'] or '(boş)'}\n")
        lines.append("**Gün 2**")
        lines.append(f"- **Sabah:** {s2['Sabah'] or '(boş)'}")
        lines.append(f"- **Öğle:** {s2['Öğle'] or '(boş)'}")
        lines.append(f"- **Akşam:** {s2['Akşam'] or '(boş)'}\n")
        if chosen_categories:
            lines.append(f"**Filtreler:** {', '.join(chosen_categories)}")
        if extra_hint:
            lines.append(f"**Not:** İpucunu dikkate aldım → {extra_hint}\n")
        lines.append("**Kaynaklar**")
        for s in sources[:5]:
            lines.append(s)
        return "\n".join(lines)

    except Exception as e:
        return f"> ⚠️ Plan oluşturulurken hata oluştu:\n```\n{repr(e)}\n```"

# --- UI ---
theme = gr.themes.Soft(primary_hue="orange")
with gr.Blocks(title="Travel Mini-Planner (Offline RAG)", theme=theme) as demo:
    gr.Markdown("## Travel Mini-Planner (Offline RAG) — İstanbul / 2 Gün")
    with gr.Row():
        cats = gr.CheckboxGroup(["müze","yürüyüş","yemek"], value=["müze"], label="Kategori filtreleri")
        hint = gr.Textbox(label="Mini ipucu (opsiyonel)", placeholder="örn. 'Ayasofya sabah olsun' | 'Topkapı 2. gün öğle'")
    btn = gr.Button("Plan Oluştur", variant="primary")
    out = gr.Markdown()
    btn.click(build_plan, inputs=[cats, hint], outputs=[out])

# Hücreyi bloklamadan çalıştır:
demo.queue().launch(share=True, inline=False, show_error=True, debug=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a867b096d21041dc81.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# === Hücre 14c — app.py’yi yaz + otomatik başlat (paylaşımlı link ile) ===
import os, re, glob, textwrap, socket, subprocess, sys, difflib

# Proje ve veri yolların
PROJECT_DIR = "/content/drive/MyDrive/Colab Notebooks/zey second akbank proje"
DATA_DIR    = f"{PROJECT_DIR}/data/istanbul"
os.makedirs(PROJECT_DIR, exist_ok=True)

# Port temizliği (7860-7890 arası)
def kill_in_use_ports(ports=range(7860, 7891)):
    for p in ports:
        try:
            subprocess.run(["fuser", "-k", f"{p}/tcp"], check=False,
                           stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        except Exception:
            pass

def find_free_port(start=7860, end=7890):
    for p in range(start, end + 1):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            try:
                s.bind(("0.0.0.0", p))
                return p
            except OSError:
                continue
    return 7860

kill_in_use_ports()
port = find_free_port()

# --- app.py içeriği (robust) ---
app_py = r'''
import os, re, glob, difflib, socket
import gradio as gr

# --- Ayarlar ---
BASE = os.environ.get("IST_TRAVEL_DATA",
                      "/content/drive/MyDrive/Colab Notebooks/zey second akbank proje/data/istanbul")

# --- Yardımcılar ---
STOPWORDS = {"olsun","lütfen","lutfen","biraz","bir","şu","bu","şöyle","böyle","gibi","ve","ya","ile","da","de"}

def _norm(s: str) -> str:
    s = (s or "").casefold()
    s = re.sub(r"[^a-zçğıöşü0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def detect_slot_from_hint(hint_norm: str) -> str | None:
    words = set(hint_norm.split())
    if "sabah" in words: return "Sabah"
    if "öğle" in words or "ogle" in words: return "Öğle"
    if "akşam" in words or "aksam" in words: return "Akşam"
    return None

def detect_day_from_hint(hint_norm: str) -> int | None:
    # ikinci gün / 2. gün / gün 2
    if re.search(r"\b(ikinci|2\.?|2)\s*g[uü]n\b|\bg[uü]n\s*2\b", hint_norm):
        return 2
    # birinci gün / 1. gün / ilk gün / gün 1
    if re.search(r"\b(birinci|ilk|1\.?|1)\s*g[uü]n\b|\bg[uü]n\s*1\b", hint_norm):
        return 1
    return None

def pick_place_from_hint(hint: str, candidates: set[str]) -> str | None:
    hn = _norm(hint)
    core = " ".join(w for w in hn.split() if w not in STOPWORDS) or hn
    best, best_score = None, 0.0
    for p in candidates:
        pn = _norm(p)
        if core and (core in pn or pn in core):
            score = 1.0
        else:
            score = difflib.SequenceMatcher(None, core, pn).ratio()
        if score > best_score:
            best, best_score = p, score
    return best if best_score >= 0.45 else None

def _pin_to_slot(slots: dict, place: str, slot_name: str) -> dict:
    # diğer slotlardan çıkar
    for k in ("Sabah","Öğle","Akşam"):
        if slots.get(k):
            items = [x.strip() for x in slots[k].split(",") if x.strip()]
            items = [x for x in items if x.lower() != place.lower()]
            slots[k] = ", ".join(items)
    # hedef slotun başına koy
    items = [x.strip() for x in (slots.get(slot_name) or "").split(",") if x.strip()]
    items = [place] + [x for x in items if x.lower() != place.lower()]
    slots[slot_name] = ", ".join(items)
    return slots

# --- Veri yükle ---
pattern_title = re.compile(r"^##\s*\[[^\]]*\]\s*([^\n]+)", re.MULTILINE)
score_pattern = re.compile(r"\b(müze|sultanahmet|topkapı|ayasofya|arkeoloji|yerebatan|gülhane|kapalı|galata|dolmabahçe)\b",
                           re.IGNORECASE)

md_files = sorted(glob.glob(os.path.join(BASE, "*.md")))
if not md_files:
    raise FileNotFoundError(f"Markdown bulunamadı: {BASE} altında *.md bekliyordum.")

place_names = set()
file_hits = []
for fp in md_files:
    with open(fp, "r", encoding="utf-8") as f:
        txt = f.read()
    for m in pattern_title.finditer(txt):
        title = re.sub(r"\s+", " ", m.group(1).strip())
        place_names.add(title)
    sc = len(score_pattern.findall(txt))
    file_hits.append((sc, os.path.basename(fp)))
file_hits.sort(reverse=True, key=lambda x: x[0])
sources = [f"[Kaynak {i}] {name} (skor={sc})" for i, (sc, name) in enumerate(file_hits, 1)]

# --- Varsayılan tercih listeleri ---
preferred_day1 = [
    "Ayasofya Camii",
    "Topkapı Sarayı Müzesi",
    "İstanbul Arkeoloji Müzeleri",
    "Gülhane Parkı",
    "Yerebatan Sarnıcı",
    "Sultan Ahmet Camii",
    "Sultanahmet Meydanı",
]
preferred_day2 = [
    "Türk ve İslam Eserleri Müzesi",
    "Kapalı Çarşı (Grand Bazaar)",
    "Galata Kulesi",
    "Dolmabahçe Sarayı",
    "Taksim Meydanı",
]

# Kategori eşlemesi (manuel, sade)
CATEGORY_MAP = {
    "müze": [
        "Topkapı Sarayı Müzesi",
        "İstanbul Arkeoloji Müzeleri",
        "Türk ve İslam Eserleri Müzesi",
        "Yerebatan Sarnıcı",
        "Ayasofya Camii",
        "Dolmabahçe Sarayı",
        "Galata Kulesi",
    ],
    "yürüyüş": [
        "Gülhane Parkı",
        "Sultanahmet Meydanı",
        "Taksim Meydanı",
        "Kapalı Çarşı (Grand Bazaar)",
    ],
    "yemek": [
        "Kapalı Çarşı (Grand Bazaar)",
        "Sultanahmet Meydanı",
        "Taksim Meydanı",
    ],
}

def fuzzy_pick(candidates, available):
    picked = []
    for cand in candidates:
        if cand in available:
            picked.append(cand); continue
        alt = [p for p in available if cand.lower() in p.lower() or p.lower() in cand.lower()]
        if alt: picked.append(alt[0])
    seen, uniq = set(), []
    for p in picked:
        if p not in seen:
            uniq.append(p); seen.add(p)
    return uniq

def apply_filters(seq, chosen):
    if not chosen: return seq
    allowed = set()
    for c in chosen:
        allowed.update(CATEGORY_MAP.get(c, []))
    out = []
    for item in seq:
        if any(item.lower() in a.lower() or a.lower() in item.lower() for a in allowed):
            out.append(item)
    return out or seq

# --- Plan oluşturucu ---
def build_plan(chosen_categories, extra_hint):
    try:
        chosen_categories = list(chosen_categories) if chosen_categories else []
        extra_hint = (extra_hint or "").strip()

        hint_norm    = _norm(extra_hint)
        slot_hint    = detect_slot_from_hint(hint_norm)   # "Sabah" / "Öğle" / "Akşam" / None
        day_hint     = detect_day_from_hint(hint_norm)    # 1 / 2 / None
        pinned_place = pick_place_from_hint(extra_hint, place_names) if extra_hint else None

        d1 = fuzzy_pick(preferred_day1, place_names)
        d2 = fuzzy_pick(preferred_day2, place_names)

        def apply_with_pin(seq):
            if not chosen_categories:
                return seq
            kept = apply_filters(seq, chosen_categories)
            if pinned_place and pinned_place in seq and pinned_place not in kept:
                kept = [pinned_place] + [x for x in kept if x != pinned_place]
            return kept

        d1 = apply_with_pin(d1)
        d2 = apply_with_pin(d2)

        if pinned_place:
            if day_hint == 2:
                d1 = [x for x in d1 if x != pinned_place]
                if pinned_place in d2:
                    d2 = [pinned_place] + [x for x in d2 if x != pinned_place]
                else:
                    d2 = [pinned_place] + d2
            else:
                d2 = [x for x in d2 if x != pinned_place]
                if pinned_place in d1:
                    d1 = [pinned_place] + [x for x in d1 if x != pinned_place]
                else:
                    d1 = [pinned_place] + d1

        def to_slots(seq):
            slots = {"Sabah": "", "Öğle": "", "Akşam": ""}
            if not seq: return slots
            if len(seq) == 1:
                slots["Sabah"] = seq[0]
            elif len(seq) == 2:
                slots["Sabah"], slots["Öğle"] = seq
            else:
                third = max(1, len(seq)//3)
                slots["Sabah"] = ", ".join(seq[:third])
                slots["Öğle"]  = ", ".join(seq[third:2*third]) if len(seq) >= 2*third else ""
                slots["Akşam"] = ", ".join(seq[2*third:]) if len(seq) > 2*third else ""
            return slots

        s1, s2 = to_slots(d1), to_slots(d2)

        if pinned_place:
            target = s2 if day_hint == 2 else s1
            target_slot = slot_hint or "Sabah"
            _pin_to_slot(target, pinned_place, target_slot)

        lines = []
        lines.append("**Gün 1**")
        lines.append(f"- **Sabah:** {s1['Sabah'] or '(boş)'}")
        lines.append(f"- **Öğle:** {s1['Öğle'] or '(boş)'}")
        lines.append(f"- **Akşam:** {s1['Akşam'] or '(boş)'}\n")
        lines.append("**Gün 2**")
        lines.append(f"- **Sabah:** {s2['Sabah'] or '(boş)'}")
        lines.append(f"- **Öğle:** {s2['Öğle'] or '(boş)'}")
        lines.append(f"- **Akşam:** {s2['Akşam'] or '(boş)'}\n")
        if chosen_categories:
            lines.append(f"**Filtreler:** {', '.join(chosen_categories)}")
        if extra_hint:
            lines.append(f"**Not:** İpucunu dikkate aldım → {extra_hint}\n")
        lines.append("**Kaynaklar**")
        for s in sources[:5]:
            lines.append(s)
        return "\n".join(lines)

    except Exception as e:
        return f"> ⚠️ Plan oluşturulurken hata oluştu:\n```\n{repr(e)}\n```"

# --- UI ---
theme = gr.themes.Soft(primary_hue="orange")
with gr.Blocks(title="Travel Mini-Planner (Offline RAG)", theme=theme) as demo:
    gr.Markdown("## Travel Mini-Planner (Offline RAG) — İstanbul / 2 Gün")
    with gr.Row():
        cats = gr.CheckboxGroup(["müze","yürüyüş","yemek"], value=["müze"], label="Kategori filtreleri")
        hint = gr.Textbox(label="Mini ipucu (opsiyonel)",
                          placeholder="örn. 'Ayasofya sabah olsun' | 'Topkapı 2. gün öğle'")
    btn = gr.Button("Plan Oluştur", variant="primary")
    out = gr.Markdown()
    btn.click(build_plan, inputs=[cats, hint], outputs=[out])

# --- LAUNCH (sağlam) ---
def _find_free_port(start=7860, end=7890):
    for p in range(start, end+1):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            try:
                s.bind(("0.0.0.0", p))
                return p
            except OSError:
                continue
    return 7860  # fallback

if __name__ == "__main__":
    try:
        gr.close_all()
    except Exception:
        pass
    demo.queue().launch(
        share=True,
        server_name="0.0.0.0",
        server_port=_find_free_port(),
        inline=False,
        debug=False,
        show_error=True
    )
'''

# app.py’yi yaz
with open(os.path.join(PROJECT_DIR, "app.py"), "w", encoding="utf-8") as f:
    f.write(textwrap.dedent(app_py).strip() + "\n")

print("✅ app.py yazıldı:", os.path.join(PROJECT_DIR, "app.py"))
print("📁 Veri klasörü:", DATA_DIR)

# Ortam değişkeni (veri yolu)
os.environ["IST_TRAVEL_DATA"] = DATA_DIR

# Eski Gradio/app.py süreçlerini kapat
try:
    import gradio as gr
    gr.close_all()
except Exception:
    pass
subprocess.run(["pkill", "-f", "app.py|gradio"], check=False)

# Proje klasörüne geç ve ÖN PLANDA çalıştır (çıktıda gradio.live linki görünür)
%cd "/content/drive/MyDrive/Colab Notebooks/zey second akbank proje"
!python -u app.py


✅ app.py yazıldı: /content/drive/MyDrive/Colab Notebooks/zey second akbank proje/app.py
📁 Veri klasörü: /content/drive/MyDrive/Colab Notebooks/zey second akbank proje/data/istanbul
/content/drive/MyDrive/Colab Notebooks/zey second akbank proje
Running on local URL:  http://0.0.0.0:7860
Running on public URL: https://9de61b3c7cdbbed131.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2709, in block_thread
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/drive/MyDrive/Colab Notebooks/zey second akbank proje/app.py", line 258, in <module>
    demo.queue().launch(
  File "/usr/local/lib/pyth

In [ ]:
# README.md içeriğini oluştur
readme_content = """
# İstanbul Mini-Gezi Planlayıcı

## Proje Amacı
Bu proje, İstanbul'daki gezilecek yerlerin planlanmasını kolaylaştıran bir araç sunmaktadır. Kullanıcılar, gezi kategorilerini (müze, yürüyüş, yemek vb.) seçebilir ve sabah, öğle, akşam gibi zaman dilimlerine göre kişiselleştirilmiş bir gezi planı alabilirler.

## Veri Seti Hakkında
Proje, **Kaggle** üzerinde bulunan `ouzcanmaden/places-to-visit-in-istanbul` veri setini kullanmaktadır. Bu veri seti, İstanbul’daki gezilecek yerlerin isimleri, kategorileri ve açıklamalarını içermektedir. Veri seti, CSV formatından Markdown formatına dönüştürülerek projemizde kullanılmaktadır.

## Kullanılan Yöntemler
- **Veri Temizleme ve İşleme:** Markdown dosyasındaki başlıklardan yer adları çıkarılarak kullanıcıya önerilen yerler sunulmuştur.
- **Gradio:** Web arayüzü oluşturmak için kullanılmıştır. Kullanıcılar, gezi kategorileri ve ipuçları ile planlama yapabilir.
- **Regex:** Yer adlarını doğru şekilde eşleştirmek için kullanılmıştır.

## Elde Edilen Sonuçlar
- Kullanıcılar, müze, yürüyüş ve yemek gibi kategorilerle filtreleme yaparak, İstanbul’daki gezilecek yerleri sabah/öğle/akşam dilimlerine yerleştirerek iki günlük bir plan oluşturabilmektedir.
- Uygulama, kullanıcıya kişiselleştirilmiş öneriler sunmaktadır.

## Kullanım
1. **Kategoriler Seçin:** Müze, yürüyüş veya yemek kategorilerinden birini seçin.
2. **Mini İpucu (Opsiyonel):** Gezi planınızı özelleştirmek için mini ipuçları ekleyin. (Örneğin: "Ayasofya sabah olsun")
3. **Plan Oluştur:** "Plan Oluştur" butonuna tıklayarak gezi planınızı oluşturun.

## Web Linki
Proje uygulamanıza [Gradio Web Uygulaması](https://9de61b3c7cdbbed131.gradio.live) üzerinden erişebilirsiniz (72 saat süreyle açık kalıyor).

"""

# README.md dosyasını Drive'a kaydet
readme_path = "/content/drive/MyDrive/Colab Notebooks/zey second akbank proje/README.md"

# Dosyayı yaz
with open(readme_path, "w", encoding="utf-8") as file:
    file.write(readme_content)

print(f"✅ README.md dosyası başarıyla oluşturuldu: {readme_path}")


✅ README.md dosyası başarıyla oluşturuldu: /content/drive/MyDrive/Colab Notebooks/zey second akbank proje/README.md


In [10]:
import nbformat

# Notebook dosyasını yükleyin
file_path = '/content/drive/MyDrive/Colab Notebooks/zey second akbank proje/code.ipynb'  # Dosya yolunu güncelleyin
with open(file_path, 'r') as f:
    notebook = nbformat.read(f, as_version=4)

# 'widgets' metadata kısmını kontrol et ve temizle
if 'widgets' in notebook['metadata']:
    del notebook['metadata']['widgets']

# Dosyayı tekrar kaydedin
with open(file_path, 'w') as f:
    nbformat.write(notebook, f)

print("Metadata temizlendi ve kaydedildi.")


Metadata temizlendi ve kaydedildi.
